In [2]:
import pandas as pd
#read csv file
customers_data = pd.read_csv('../data/raw/olist_customers_dataset.csv')
order_items_data = pd.read_csv('../data/raw/olist_order_items_dataset.csv')
orders_data = pd.read_csv('../data/raw/olist_orders_dataset.csv', parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ])
products_data = pd.read_csv('../data/raw/olist_products_dataset.csv')
category_data = pd.read_csv('../data/raw/product_category_name_translation.csv')

In [3]:
# filter only delivered orders
delivered_orders = orders_data[orders_data['order_status'] == 'delivered']
# drop orders without purchase timestamp
delivered_orders = delivered_orders.dropna(subset=['order_purchase_timestamp'])
# reduce to only necessary columns
delivered_orders = delivered_orders[['order_id', 'customer_id', 'order_purchase_timestamp']]
order_items_data = order_items_data[['order_id', 'product_id', 'price']]
products_data = products_data[['product_id', 'product_category_name']]
customers_data = customers_data[['customer_id','customer_unique_id', 'customer_state']]



In [4]:
# create fact sales table

fact_sales = delivered_orders.merge(order_items_data, on='order_id', how='left')

fact_sales = fact_sales.merge(products_data, on='product_id', how='left')

fact_sales = fact_sales.merge(customers_data, on='customer_id', how='left')

fact_sales = fact_sales.merge(category_data, left_on='product_category_name',right_on='product_category_name', how='left')

# fill missing category names
fact_sales["product_category_name_english"] = (
    fact_sales["product_category_name_english"]
    .fillna("unknown")
)

# feature engineering
fact_sales['order_month'] = fact_sales['order_purchase_timestamp'].dt.to_period('M').astype(str)
fact_sales['revenue'] = fact_sales['price']


In [5]:
# save fact sales table
fact_sales.to_csv(
    "../data/processed/fact_sales.csv",
    index=False
)
print(fact_sales.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110197 entries, 0 to 110196
Data columns (total 11 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       110197 non-null  object        
 1   customer_id                    110197 non-null  object        
 2   order_purchase_timestamp       110197 non-null  datetime64[ns]
 3   product_id                     110197 non-null  object        
 4   price                          110197 non-null  float64       
 5   product_category_name          108660 non-null  object        
 6   customer_unique_id             110197 non-null  object        
 7   customer_state                 110197 non-null  object        
 8   product_category_name_english  110197 non-null  object        
 9   order_month                    110197 non-null  object        
 10  revenue                        110197 non-null  float64       
dtype